# CoT (Chain of Thought)

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
import openai
client = openai.OpenAI()

### 일반 프롬프트 vs Zero-shot CoT

In [9]:
# 일반 프롬프트
prompt = '123 x 31은 얼마인가요?'


# Zero-shot CoT 적용 프롬프트
cot_prompt = '단계별로 생각해보세요: 123 x 31은 얼마인가요?'

res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": prompt}]
    
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [6]:
res.choices[0].message.content

'123 x 31 = 3813'

In [10]:
print(cot_res.choices[0].message.content)

123 x 31의 계산을 단계별로 나눠보겠습니다.

1단계: 123 x 1 = 123
2단계: 123 x 30 = 3690

그럼 이제 이 둘을 더해봅시다.

123 + 3690 = 3813

따라서, 123 x 31 = 3813입니다.


### Few-shot CoT

In [11]:
few_shot_cot = """
Q: 123 x 31 은 얼마인가요?
A: 
1. 123를 분해합니다: 123 = 100 + 20 + 3
2. 각 항을 31과 곱합니다:
   - 100 x 31 = 3100
   - 20 x 31 = 620
   - 3 x 31 = 93

3. 이제 이 결과들을 더합니다:
   - 3100 + 620 + 93

4. 계산을 진행하면:
   - 3100 + 620 = 3720
   - 3720 + 93 = 3813 

따라서, 123 x 31 = 3813입니다.

Q: 789 x 56 은 얼마인가요?
A:
"""

res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": few_shot_cot}]
)

print(res.choices[0].message.content)

1. 789를 분해합니다: 789 = 700 + 80 + 9
2. 각 항을 56과 곱합니다:
   - 700 x 56 = 39200
   - 80 x 56 = 4480
   - 9 x 56 = 504

3. 이제 이 결과들을 더합니다:
   - 39200 + 4480 + 504

4. 계산을 진행하면:
   - 39200 + 4480 = 43680
   - 43680 + 504 = 44184 

따라서, 789 x 56 = 44184입니다.


### 논리 문제 해결

In [12]:
prompt = "앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다. 프랭크(Frank)는 데이비드(David)보다 어리다. 그렇다면 가장 어린 사람은 누구인가요?"

cot_prompt = "단계적으로 생각해서 대답해주세요: 앨리스(Alice)는 밥(Bob)보다 나이가 많다. 밥(Bob)은 찰리(Charlie)보다 나이가 많다. 데이비드(David)는 앨리스(Alice)보다 어리지만, 에밀리(Emily)보다는 많다. 찰리(Charlie)는 프랭크(Frank)보다 많다. 에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다. 프랭크(Frank)는 데이비드(David)보다 어리다. 그렇다면 가장 어린 사람은 누구인가요?"

In [13]:
res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": prompt}]
    
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [14]:
print(res.choices[0].message.content)

프랭크(Frank)입니다.


In [15]:
print(cot_res.choices[0].message.content)

가장 어린 사람을 찾아보겠습니다.

1. 앨리스(Alice)는 밥(Bob)보다 많으므로 밥(Bob)은 앨리스(Alice)보다 어리다.
2. 밥(Bob)은 찰리(Charlie)보다 나이가 많으므로 찰리(Charlie)는 밥(Bob)보다 어리다.
3. 따라서 앨리스(Alice)와 밥(Bob), 찰리(Charlie) 중 가장 어린 사람은 찰리(Charlie)라고 할 수 있다.
4. 다음으로, 데이비드(David)는 앨리스(Alice)보다 어리지만 에밀리(Emily)보다는 많다고 했으므로, 데이비드(David)는 찰리(Charlie)보다 나이가 많습니다.
5. 에밀리(Emily)는 찰리(Charlie)보다 나이가 많지만, 밥(Bob)보다는 어리다고 했으므로, 에밀리(Emily)도 찰리(Charlie)보다 나이가 많습니다. 
6. 프랭크(Frank)는 데이비드(David)보다 어리다고 했으므로, 프랭크(Frank)는 데이비드(David), 앨리스(Alice), 밥(Bob), 에밀리(Emily), 찰리(Charlie) 모두보다 어립니다.

따라서 가장 어린 사람은 프랭크(Frank)입니다.


### 코드 디버깅

In [16]:
prompt = """
다음과 같이 코드를 작성했더니 에러가 발생합니다. 왜일까요?

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

cot_prompt = """
다음과 같이 코드를 작성했더니 에러가 발생합니다.
실수가 무엇인지 찾아서 단계별로 설명해주세요

def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
"""

In [19]:
res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": prompt}]
    
)

cot_res = client.chat.completions.create(
    model='gpt-4',
    messages=[{"role": "user", "content": cot_prompt}]
)

In [20]:
print(res.choices[0].message.content)

코드의 들여쓰기(indentation)가 잘못되어 에러가 발생했습니다. 들여쓰기는 파이썬에서 코드의 블록을 구분하는 매우 중요한 요소입니다. 

함수의 본문인 `return a + b` 부분이 함수의 정의 안에 속해야 하므로, 이 부분을 들여쓰기 해야 합니다.

아래 코드처럼 수정해야 합니다:

```python
def add_numbers(a, b):
  return a + b

print(add_numbers(1, 2))
```


In [21]:
print(cot_res.choices[0].message.content)

다음과 같이 코드를 작성하였을 때 들여쓰기 에러(IndentationError)가 발생합니다.

```python
def add_numbers(a, b):
return a + b

print(add_numbers(1, 2))
```

위의 문제는 파이썬이 들여쓰기를 강제하는 언어이기 때문인데, 함수의 본문 부분이 들여쓰기 되지 않았기 때문입니다. 

수정 방법은 다음과 같습니다.

```python
def add_numbers(a, b):
    return a + b

print(add_numbers(1, 2))
```

위의 코드처럼 함수 본문인 return a + b 명령어 앞에 빈 칸 4개(또는 탭 1개, 환경에 따라 다를 수 있음)를 추가하여 들여쓰기를 해주면 되는데, 이렇게 들여쓰기를 함으로써 add_numbers 함수의 정의 부분임을 파이썬에게 알려주게 됩니다.
위와 같이 수정하면 에러 없이 정상적으로 코드가 실행되며, 결과로 3이 출력되게 됩니다.
